In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt
import seaborn as sns

In [5]:
'''
using 3 types of material, measurement points intake, after a, after b

'''

def dynamics(A, B, x, u):
    x_new = A @ x + B @ u
    x_new[x_new < 0] = 0  #clip material quantities to zero
    return x_new
    
    
    
# A = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
# B = np.array([[0, 0], [0, 0], [0,0]])|
# u = np.array([0,0]).T
# x = np.array([2, 0, 0]).T

In [6]:
import matplotlib.pyplot as plt
%matplotlib qt

exes = [0, 1, 2, 3, 4, 5, 6, 7, 8]
ys = [0, 1.25, 5, 11.25, 20, 33, 52.75, 78.6875, 111]

plt.plot(exes, ys)

In [76]:
def problem_constructor_LP(num_materials = 4, set_params = False):
    
    '''
    x order: (assuming 4 materials)
    initial state (4)
    stage 1 (4)
    stage 2 (4)
    stage 3/out material 4 (4) 
    effort mat 1
    effort mat 2
    effort mat 3
    belt speed
    '''
    nx = 0
    nu = 0
    # nx += num_materials  # compisition of feed pile, normalized to 1
    nx += num_materials ** 2 # quantity of each material at each stage (intake, stage 1, stage ..., stage num_material-1)
    nx += num_materials - 1 # effort at each positive sort station
    # nx += 1 # belt speed (really, how much to take from feed pile each time step)
    
    # nu = num_materials - 1 # how much to increase or decrease effort at each positive sort station
    nu += 1 #how much to increase or decrease belt speed
    
    x = np.zeros((nx))
    u = np.zeros((nu))
    
    A = np.zeros((nx, nx))
    B = np.zeros((nx, nu))
    Q_vect = np.zeros((nx))
    R_vect = np.ones((nu))
    
    
    ## assemble a and b matrices
    
    priceA = 13 #$/ton
    priceB = 10 #$/ton
    priceC = 20 #$/ton
    priceD = 4 #$/ton
    
    Q_vect[-1] = priceC
    Q_vect[-2] = priceB
    Q_vect[-3] = priceA
    Q_vect[-4] = priceD-priceA
    Q_vect[-5] = priceD-priceB
    Q_vect[-6] = priceD-priceC
    Q_vect[-7] = priceD
    
    # R_vect *= -1
    # R_vect[-1] = 0
    
    # Q = np.diag(Q_vect)
    # R = np.diag(R_vect)
    
    
    
    
    feedA = 0.5
    feedB = 1
    feedC = .8
    feedD = 0.2
    # feedA = 1
    # feedB = 1
    # feedC = 1
    # feedD = 1
    
    
    sum = feedA + feedB + feedC + feedD
    feedA = feedA/sum
    feedB = feedB/sum
    feedC = feedC/sum
    feedD = feedD/sum
    
    B[0, -1] = feedA
    B[1, -1] = feedB
    B[2, -1] = feedC
    B[3, -1] = feedD
    
    
    
    # top part of last column of b encodes feed pile ratios, u[-1] is how much feed pile to put in first few rows of x
    
    
    # A[-1, -1] = 1 # belt speed no state transition
    # B[-1, -1] = 1 # belt speed increase or decrease equal to last element of u
    
    ## feed pile stays the same
    # for i in range(num_materials):
    #     A[i, i] = 1
    
    
    for i in range(num_materials-1):
        
        # move stuff down the conveyor belt (first station draws from feed pile)
        row_start = (i+1)*num_materials
        col_start = i*num_materials
        for j in range(num_materials):
            Ar = row_start+j
            Ac = col_start+j
            A[Ar, Ac] = 1
        # remove material from flow at sorting stations
        col_sort = num_materials**2 + i
        A[row_start + i, col_sort] = -1
        A[-1*(i+1), -1*(i+1)] = 1
        
    
    # if num_materials == 4 and set_params == True:
    #     print()
    # else:
    #     for i in range(num_materials):
    #         B[i, -1] = np.random()
    #         print('feed pile material a ')
            
    # r_start = num_materials **2 -1
            
    # for i in range(num_materials-1):
        
    #     # adjust sorting effrots as per controls
        
    #     B[r_start+i, i] = 1
    
    return A, B, x, u, Q_vect, R_vect

In [90]:
def problem_constructor_LP(num_materials = 4, param = 1, d=1, set_params = False):
    
    '''
    x order: (assuming 4 materials)
    initial state (4)
    stage 1 (4)
    stage 2 (4)
    stage 3/out material 4 (4) 
    effort mat 1
    effort mat 2
    effort mat 3
    belt speed
    '''
    nx = 0
    nu = 0
    # nx += num_materials  # compisition of feed pile, normalized to 1
    nx += num_materials ** 2 # quantity of each material at each stage (intake, stage 1, stage ..., stage num_material-1)
    nx += num_materials - 1 # effort at each positive sort station
    # nx += 1 # belt speed (really, how much to take from feed pile each time step)
    
    # nu = num_materials - 1 # how much to increase or decrease effort at each positive sort station
    nu += 1 #how much to increase or decrease belt speed
    
    x = np.zeros((nx))
    u = np.zeros((nu))
    
    A = np.zeros((nx, nx))
    B = np.zeros((nx, nu))
    Q_vect_plenty = np.zeros((nx))
    R_vect = np.ones((nu))
    Q_vect_tapped = np.zeros((nx))
    
    
    ## assemble a and b matrices
    
    priceA = 66 #$/ton HDPE
    priceB = 13 #$/ton pet
    priceC = 5 #$/ton pp
    priceD = 1 #$/ton 3-7 comingled
    
    # Q_vect_plenty[-1] = priceC
    # Q_vect_plenty[-2] = priceB
    # Q_vect_plenty[-3] = priceA
    # Q_vect_plenty[-4] = priceD-priceA
    # Q_vect_plenty[-5] = priceD-priceB
    # Q_vect_plenty[-6] = priceD-priceC
    # Q_vect_plenty[-7] = priceD
    
    Q_vect_tapped[0] = priceA
    Q_vect_tapped[5] = priceB
    Q_vect_tapped[10] = priceC
    Q_vect_tapped[-4] = priceD
    Q_vect_tapped[-5] = priceD-priceC
    Q_vect_tapped[-6] = priceD-priceB
    Q_vect_tapped[-7] = priceD - priceA
    
    # R_vect *= -1
    # R_vect[-1] = 0
    
    # Q = np.diag(Q_vect)
    # R = np.diag(R_vect)
    
    
    
    
    # feedA = 0.5
    # feedB = 1
    # feedC = .8
    # feedD = 0.2
    feedA = 1 * param
    feedB = 1
    feedC = 1
    feedD = 1
    
    
    sum = feedA + feedB + feedC + feedD
    feedA = feedA/sum
    feedB = feedB/sum
    feedC = feedC/sum
    feedD = feedD/sum
    
    B[0, -1] = feedA
    B[1, -1] = feedB
    B[2, -1] = feedC
    B[3, -1] = feedD
    
    
    
    # top part of last column of b encodes feed pile ratios, u[-1] is how much feed pile to put in first few rows of x
    
    
    # A[-1, -1] = 1 # belt speed no state transition
    # B[-1, -1] = 1 # belt speed increase or decrease equal to last element of u
    
    ## feed pile stays the same
    # for i in range(num_materials):
    #     A[i, i] = 1
    
    
    for i in range(num_materials-1):
        
        # move stuff down the conveyor belt (first station draws from feed pile)
        row_start = (i+1)*num_materials
        col_start = i*num_materials
        for j in range(num_materials):
            Ar = row_start+j
            Ac = col_start+j
            A[Ar, Ac] = 1
        # remove material from flow at sorting stations
        col_sort = num_materials**2 + i
        A[row_start + i, col_sort] = -1
        A[-1*(i+1), -1*(i+1)] = 1
        
    
    # if num_materials == 4 and set_params == True:
    #     print()
    # else:
    #     for i in range(num_materials):
    #         B[i, -1] = np.random()
    #         print('feed pile material a ')
            
    # r_start = num_materials **2 -1
            
    # for i in range(num_materials-1):
        
    #     # adjust sorting effrots as per controls
        
    #     B[r_start+i, i] = 1
    
    return A, B, x, u, Q_vect_tapped, R_vect

In [55]:
A, B, x, u, Q_t, R = problem_constructor_LP(4)

# initial conditions:
x[-3] = 1
x[-2] = 1
x[-1] = 1
R[-1] = -0.1

u[-1] = 8

In [85]:
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

In [86]:
matprint(A)

0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   0   0   0  
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   0   0   0  
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   0   0   0  
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   0   0   0  
1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  -1   0   0  
0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0   0   0   0  
0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0   0   0   0  
0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0   0   0   0  
0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0   0   0   0  
0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0   0  -1   0  
0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0   0   0   0  
0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0   0   0   0  
0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0   0   0   0  
0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0   0   0   0  
0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0   0   0  -1  
0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0   0   0   0  
0  0  0  0  0  0  0  0  

In [101]:
p = 0.25

A, B, x, u, Q_t, R = problem_constructor_LP(4, param = p, d = 8)

# initial conditions:
x[-3] = 1
x[-2] = 1
x[-1] = 1
R[-1] = -0.1


print(x)


# u[0] =-0.25
# u[1] = 0.25

# u[0] =0
# u[1] = 0
u[0] = 1
exes = np.linspace(1, 10, num=1000)
ys = np.zeros_like(exes)
k=0
not_switched = True
x_mask = 1000000000*np.ones_like(x)
x_mask[0] = 1
x_mask[5] = 1
x_mask[10] = 1
for j in exes:
    
    u[0]=j
    for i in range(100):
        x = dynamics(A, B, x, u)
    x_clipped = np.minimum(x, x_mask)
    # obj = np.min(np.array([Q_p@x, Q_t@x])) + R@u
    # print(x_clipped[0], x_clipped[5], x_clipped[10])
    obj = Q_t@x_clipped
    ys[k] = obj
    # print(j)
    # print(obj)
    # print(x)
    # if (np.min(np.array([Q_p@x, Q_t@x])) != Q_t@x) and not_switched:
    #     print('Switched!: ', j)
    #     not_switched = False
    k+=1
lab_str = "Ratio of HDPE to mean: " + str(p)
plt.plot(exes, ys, label=lab_str)
plt.legend()
plt.xlabel('Belt Speed')
plt.ylabel('Steady State Objective Function Value')
plt.show()
print(obj)
print(x)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
38.61538461538461
[0.76923077 3.07692308 3.07692308 3.07692308 0.         3.07692308
 3.07692308 3.07692308 0.         2.07692308 3.07692308 3.07692308
 0.         2.07692308 2.07692308 3.07692308 1.         1.
 1.        ]


In [104]:
params = np.linspace(-1, 2, 20)



for d in range(1):
    best_speeds = np.zeros_like(params)
    exes_log = 10 ** params
    index = 0
    exes = np.linspace(1, 10, num=50)
    mat = np.zeros_like(exes)
    for p in params:
        
        
        A, B, x, u, Q_t, R = problem_constructor_LP(4, p, 4)

        # initial conditions:
        x[-3] = 1
        x[-2] = 1
        x[-1] = 1
        R[-1] = -0.1


        # print(x)

        # u[0] =-0.25
        # u[1] = 0.25

        # u[0] =0
        # u[1] = 0
        u[0] = 1
        
        ys = np.zeros_like(exes)
        k=0
        not_switched = True
        x_mask = 1000000000*np.ones_like(x)
        x_mask[0] = 1
        x_mask[5] = 1
        x_mask[10] = 1
        for j in exes:
            
            u[0]=j
            for i in range(100):
                x = dynamics(A, B, x, u)
            x_clipped = np.minimum(x, x_mask)
            # obj = np.min(np.array([Q_p@x, Q_t@x])) + R@u
            obj = Q_t@x_clipped
            ys[k] = obj
            # print(j)
            # print(obj)
            # print(x)
            # if (np.min(np.array([Q_p@x, Q_t@x])) != Q_t@x) and not_switched:
            #     print('Switched!: ', j)
            #     not_switched = False
            k+=1
        mat = np.vstack((mat, ys))
        best_y = np.max(ys)
        best_x = exes[np.where(ys == best_y)]
        best_speeds[index] = best_x
        index +=1
        lab_str = "Price of HDPE: $" + str(np.round(10**p, 2)) 
        # plt.plot(exes, ys, label=lab_str)
        plt.plot(exes_log, best_speeds, label = lab_str)
    
plt.legend()
plt.title("Changing material prices does not change optimal belt speed")
plt.ylabel("Steady-State Objective Function Value")
plt.xlabel('Belt Speed')
plt.show()    
# best_speeds = np.zeros_like(params)
# index = 0
# for p in params:
    
    
#     A, B, x, u, Q_p, Q_t, R = problem_constructor_LP(4, p, 10)

#     # initial conditions:
#     x[-3] = 1
#     x[-2] = 1
#     x[-1] = 1
#     R[-1] = -0.1


#     # print(x)

#     # u[0] =-0.25
#     # u[1] = 0.25

#     # u[0] =0
#     # u[1] = 0
#     u[0] = 1
#     exes = np.linspace(1, 10, num=100)
#     ys = np.zeros_like(exes)
#     k=0
#     not_switched = True
#     x_mask = 1000000000*np.ones_like(x)
#     x_mask[0] = 1
#     x_mask[5] = 1
#     x_mask[10] = 1
#     for j in exes:
        
#         u[0]=j
#         for i in range(100):
#             x = dynamics(A, B, x, u)
#         x_clipped = np.minimum(x, x_mask)
#         # obj = np.min(np.array([Q_p@x, Q_t@x])) + R@u
#         obj = Q_t@x_clipped
#         ys[k] = obj
#         # print(j)
#         # print(obj)
#         # print(x)
#         # if (np.min(np.array([Q_p@x, Q_t@x])) != Q_t@x) and not_switched:
#         #     print('Switched!: ', j)
#         #     not_switched = False
#         k+=1
#     best_y = np.max(ys)
#     best_x = exes[np.where(ys == best_y)]
#     best_speeds[index] = best_x
#     index +=1
#     lab_str = "Price of D: $10, Price of A: $" + str(100 ** p)
#     # plt.plot(exes, ys, label = lab_str, linestyle = 'dashed')



print(obj)
print(x)

-125.0
[4. 2. 2. 2. 3. 2. 2. 2. 3. 1. 2. 2. 3. 1. 1. 2. 1. 1. 1.]


In [38]:
ax = sns.heatmap(mat, linewidth=0.5, xticklabels = np.round(exes, 2), yticklabels = np.round(exes_log, 2), cbar_kws={'label': 'Steady-State Objective Function Value'})
plt.xlabel('Belt Speed')
plt.ylabel('Quantity of HDPE in Flow')
plt.show()


In [83]:
A, B, x, u, Q_t, R = problem_constructor_LP(4, p, 4)

Q_pos = np.zeros_like(Q_t)
Q_neg = np.zeros_like(Q_t)
Q_neg_abs = np.zeros_like(Q_t)

Q_pos[np.where(Q_t > 0)] = Q_t[np.where(Q_t > 0)]
Q_neg[np.where(Q_t < 0)] = Q_t[np.where(Q_t < 0)]
Q_neg_abs[np.where(Q_t < 0)] = -Q_t[np.where(Q_t < 0)]

T = 50
X = cp.Variable((x.shape[0], T+1))
U = cp.Variable((u.shape[0], T))

In [84]:
# ((-1*Q_pos)@(cp.minimum(X[:, 0], x_mask))).curvature
# (-1 * Q_neg@X[:, 0] - Q_pos@X[:, 0]).curvature
# (X[:, 0].T @ np.diag(Q_pos) @ X[:, 0]).curvature
# (cp.quad_form(X[:, 0], np.diag(Q_neg)) + cp.quad_form(X[:, 0], np.diag(Q_pos))).curvature


rew =(cp.quad_form(X[:, 0], np.diag(Q_neg_abs)) )
# rew = Q_pos@X[:, 0]
# rew += Q_neg@X[:, 0]
rew.curvature

'CONVEX'

In [79]:
Q_neg

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0., -65., -12.,  -4.,   0.,   0.,   0.,   0.])

In [80]:

A, B, x, u, Q_t, R = problem_constructor_LP(4, p, 4)
T = 50
X = cp.Variable((x.shape[0], T+1))
U = cp.Variable((u.shape[0], T))

x_mask = 1000000000*np.ones_like(x)
x_mask[0] = 1
x_mask[5] = 1
x_mask[10] = 1

x[-3] = 1
x[-2] = 1
x[-1] = 1
R[-1] = -0.1


reward = 0
constraints = []

for t in range(T):
    reward += Q_pos@cp.minimum(X[:, t+1], x_mask)
    reward -= (-1*Q_neg)@cp.minimum(X[:, t+1], x_mask)
    constraints += [X[:, t+1] == A @ X[:, t] + B@U[:, t], X >= 0]
    
constraints += [X[:, 0] == x]

problem = cp.Problem(cp.Maximize(reward), constraints)
problem.solve(verbose=True, qcp=True)

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Apr 25 04:30:10 PM: Your problem has 1019 variables, 101 constraints, and 0 parameters.
(CVXPY) Apr 25 04:30:10 PM: It is compliant with the following grammars: 
(CVXPY) Apr 25 04:30:10 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 25 04:30:10 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.


DQCPError: The problem is not DQCP.

In [106]:
Q

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,
        4.,  4.,  4., 13., 10., 20.])

In [107]:
R


array([-1., -1., -1.,  0.])